# Imports and Constants

In [1]:
# Workflow
import random
import tqdm
import tqdm.notebook

# Computation
import numpy as np
import torch
import torch.nn as nn

# Data visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(rc={"figure.dpi":100, 'savefig.dpi':100})
sns.set_context('notebook')
random.seed(0)

torch.__version__

'1.7.1+cu92'

In [2]:
# Keys
SCENE_IDX = 'scene_idx'
P_IN = 'p_in'
V_IN = 'v_in'
P_OUT = 'p_out'
V_OUT = 'v_out'
L_IN = 'closest_lane_in'
N_IN = 'closest_norm_in'

# Header of predictions CSV file
CSV_HEADER = ['ID,'] + ['v' + str(i) + ',' for i in range(1, 60)] + ['v60', '\n']

TRAIN_TEST_RATIO = 0.99
BATCH_SIZE_TRAIN = 256
BATCH_SIZE_TEST = 256
BATCH_SIZE_VAL = 64
N_WORKERS = 2

MODEL_STATE = 'model_state_dict'
OPTIMIZER_STATE = 'optimizer_state_dict'
EPOCH_STATE = 'epoch'
LOSS_STATE = 'loss'
BATCH_STATE = 'batch'
SCHEDULER_STATE = 'scheduler_state_dict'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def get_original_data(PIN, VIN, POUT, LIN=None, NIN=None):
    # Build the dataset
    original_data = []
    if LIN is None:
        iterator = tqdm.notebook.tqdm(zip(PIN.iterrows(), VIN.iterrows(), POUT.iterrows()), total=len(PIN))
        for (i1, r1), (_, r2), (_, r3) in iterator:
            d = {
                SCENE_IDX: r1[0], 
                P_IN: r1.to_numpy()[1:].reshape(19, 2),
                V_IN: r2.to_numpy()[1:].reshape(19, 2),
                P_OUT: r3.to_numpy()[1:].reshape(30, 2)
            }
            original_data.append(d)
    else:
        iterator = tqdm.notebook.tqdm(zip(PIN.iterrows(), VIN.iterrows(), POUT.iterrows(), 
                                          LIN.iterrows(), NIN.iterrows()), total=len(PIN))
        for (i1, r1), (_, r2), (_, r3), (_, r4), (_, r5) in iterator:
            d = {
                SCENE_IDX: r1[0], 
                P_IN: r1.to_numpy()[1:].reshape(19, 2),
                V_IN: r2.to_numpy()[1:].reshape(19, 2),
                P_OUT: r3.to_numpy()[1:].reshape(30, 2),
                L_IN: r4.to_numpy()[1:].reshape(19, 2),
                N_IN: r5.to_numpy()[1:].reshape(19, 2)
            }
            original_data.append(d)
    
    return original_data

def get_train_test_data(original_data, ratio):
    # Split into train and test
    random.shuffle(original_data)
    TRAIN_SIZE = int(len(original_data) * ratio)
    train_data = original_data[:TRAIN_SIZE]
    test_data = original_data[TRAIN_SIZE:]
    return train_data, test_data
    
def get_val_data(PIN, VIN, LIN=None, NIN=None):
    val_data = []
    if LIN is None:
        iterator = tqdm.notebook.tqdm(zip(PIN.iterrows(), VIN.iterrows()), total=len(PIN))
        for (i1, r1), (_, r2) in iterator:
            d = {
                SCENE_IDX: r1[0], 
                P_IN: r1.to_numpy()[1:].reshape(19, 2),
                V_IN: r2.to_numpy()[1:].reshape(19, 2)
            }
            val_data.append(d)
    else:
        iterator = tqdm.notebook.tqdm(zip(PIN.iterrows(), VIN.iterrows(), LIN.iterrows(), NIN.iterrows()), total=len(PIN))
        for (i1, r1), (_, r2), (_, r3), (_, r4), in iterator:
            d = {
                SCENE_IDX: r1[0], 
                P_IN: r1.to_numpy()[1:].reshape(19, 2),
                V_IN: r2.to_numpy()[1:].reshape(19, 2),
                L_IN: r3.to_numpy()[1:].reshape(19, 2),
                N_IN: r4.to_numpy()[1:].reshape(19, 2)
            }
            val_data.append(d)
    return val_data

def get_data(kind, features):
    # Train/test data
    LIN = None
    NIN = None
    if kind == 'single':
        PIN = pd.read_csv('./targ_train_data/pin_train.csv')
        VIN = pd.read_csv('./targ_train_data/vin_train.csv')
        POUT = pd.read_csv('./targ_train_data/pout_train.csv')

    else:    
        # All tracked agents
        PIN = pd.read_csv('./track_train_data/pin_trainall.csv')
        VIN = pd.read_csv('./track_train_data/vin_trainall.csv')
        POUT = pd.read_csv('./track_train_data/pout_trainall.csv')
        
        if features == 'extended':
            LIN = pd.read_csv('./track_train_data/lanein.csv')
            NIN = pd.read_csv('./track_train_data/normin.csv')

        
    original_data = get_original_data(PIN, VIN, POUT, LIN, NIN)
    train_data, test_data = get_train_test_data(original_data, TRAIN_TEST_RATIO)
    
    # Validation data
    PIN = pd.read_csv('./targ_val_data/pin_val.csv')
    VIN = pd.read_csv('./targ_val_data/vin_val.csv')
    if features == 'extended':
        LIN = pd.read_csv('./targ_val_data/lanein.csv')
        NIN = pd.read_csv('./targ_val_data/normin.csv')
        
    val_data = get_val_data(PIN, VIN, LIN, NIN)
    
    return train_data, test_data, val_data

# Dataset Loading and Batching

In [4]:
class ArgoverseDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        super(ArgoverseDataset, self).__init__()
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]      

In [5]:
def collate_train_test(batch):
    """ 
    Custom collate_fn function to be used for DataLoader.    
    The input tensor is organized as 19 rows, where each row has 4 columns: px, py, vx, vy
    
    The output tensor can be organized in 2 ways:
    A) The output tensor is organized as 120 rows, where each row has 1 column. 
    Each contiguous sequence of 4 elements is px, py, vx, vy
    
    B) The input tensor is organized as 30 rows, where each row has 4 columns: px, py, vx, vy
    """    
    inp = []     
    out = []
    scene_idxs = []
    offsets = np.zeros((len(batch), 2))
    for i, scene in enumerate(batch):      
        pin, vin, pout = np.copy(scene[P_IN]), np.copy(scene[V_IN]), np.copy(scene[P_OUT])        
        # Normalize position
        xs, ys = pin[0, 0], pin[0, 1]       
        pin[:, 0] -= xs
        pin[:, 1] -= ys
        pout[:, 0] -= xs
        pout[:, 1] -= ys
        
        lin, nin = np.copy(scene[L_IN]), np.copy(scene[N_IN])
        lin[:, 0] -= xs
        lin[:, 1] -= ys
        inp_tens = np.concatenate((pin, vin, lin, nin), axis=1) 
    
#         inp_tens = np.concatenate((pin, vin), axis=1)     
        out_tens = pout
        
        inp.append(inp_tens)
        out.append(out_tens)  
        scene_idxs.append(scene[SCENE_IDX])
        offsets[i, 0], offsets[i, 1] = xs, ys

    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out, scene_idxs]

In [6]:
def collate_val(batch):
    """ 
    Custom collate_fn for validation dataset. The validation data do not contain output values.   
    The input tensor is organized as 19 rows, where each row has 4 columns: px, py, vx, vy
    """   
    inp = []
    scene_idxs = []
    offsets = np.zeros((len(batch), 2))  # start (x, y) coordinates from normalization
    
    for i, scene in enumerate(batch):          
        pin, vin = np.copy(scene[P_IN]), np.copy(scene[V_IN])
        # Normalize position
        xs, ys = pin[0, 0], pin[0, 1]      
        pin[:, 0] -= xs
        pin[:, 1] -= ys
        
        lin, nin = np.copy(scene[L_IN]), np.copy(scene[N_IN])
        lin[:, 0] -= xs
        lin[:, 1] -= ys
        inp_tens = np.concatenate((pin, vin, lin, nin), axis=1)
        
#         inp_tens = np.concatenate((pin, vin), axis=1)        
        inp.append(inp_tens)        
        scene_idxs.append(scene[SCENE_IDX])
        offsets[i, 0], offsets[i, 1] = xs, ys
    inp = torch.FloatTensor(inp)    
    return [inp, scene_idxs, offsets]

# Training Workflow

In [7]:
def train(model, device, train_loader, optimizer, epoch, train_data):
    # Set the model into training mode
    model.train()      
    # Define the loss function.
    criterion = torch.nn.MSELoss(reduction='mean')

    total_loss = 0    
    for i in range(epoch):
        iterator = tqdm.notebook.tqdm(train_loader, total=int(len(train_loader)))
        for _, batch in enumerate(iterator):
            data, target, _ = batch
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()  
            
            out = model(data)
            # Compute the loss       
            loss = torch.sqrt(criterion(out, target)) 
            total_loss += loss.item()

            # Perform backpropagation
            loss.backward()
            # Update the weights
            optimizer.step()    
            
#             scheduler.step()  # Uncomment if using CyclicLR

            # Update the progress bar for tqdm
            iterator.set_postfix(train_loss=loss.item())
            
    return (total_loss * BATCH_SIZE_TRAIN) / len(train_data)

In [8]:
def test(model, device, test_loader, test_data):
    model.eval()    
    criterion = torch.nn.MSELoss(reduction='mean') 

    iterator = tqdm.notebook.tqdm(test_loader, total=int(len(test_loader)))
    total_loss = 0    
    for _, batch in enumerate(iterator):
        data, target, _ = batch
        data, target = data.to(device), target.to(device)
        
        with torch.no_grad():
            out = model(data)            
            loss = torch.sqrt(criterion(out, target))        
            total_loss += loss.item()

            iterator.set_postfix(test_loss=loss.item())
                    
    return (total_loss * BATCH_SIZE_TEST) / len(test_data)

In [9]:
def validate(model, device, val_loader, path):
    """
    path: path to csv file to write predictions
    """
    model.eval() 
    
    # Prep the output file
    with open(path, "w") as csv_file:
        # Clear the csv file before appending data to it
        csv_file.truncate()
        # Write the header to the csv file
        csv_file.writelines(CSV_HEADER)    
    
    # Make predictions
    with open(path, "a") as pred_file:        
        iterator = tqdm.notebook.tqdm(val_loader, total=int(len(val_loader)))
        
        for _, batch in enumerate(iterator):
            data, scene_idxs, offsets = batch
            data = data.to(device) 
            
            with torch.no_grad():
                output = model(data)
                # Convert the Tensor from GPU -> CPU -> NumPy array
                np_out = output.cpu().detach().numpy()
                
                # Store only the predictions for the target agent and keep the positions, not the velocities
                batch_size = np_out.shape[0]
                
                pred = np.zeros((batch_size, 60))
                # The output should be a (30 x 2) tensor 
                # where the first two features are the out position x, y
                for i in range(batch_size):
                    # Re-scale values
                    xs, ys = offsets[i, 0], offsets[i, 1]
                    np_out[i, :, 0] += xs
                    np_out[i, :, 1] += ys
                    pred[i] = np_out[i, :, :2].flatten()                       

                # Form comma-separated string
                s = []
                for i in range(pred.shape[0]):
                    s.append(','.join([str(scene_idxs[i])] + [str(v) for v in pred[i]]) + '\n')

                # Write data to file
                pred_file.writelines(s)

# Model Initialization

In [10]:
# class ArgoNet(torch.nn.Module):
#     """
#     Neural Network class - linear regression
#     """
#     def __init__(self, device):
#         super(ArgoNet, self).__init__() 
        
#         self.device = device        
#         # Linear regression for 1 agent     
#         self.fc = nn.Linear(19 * 4, 30 * 2)   
    
#     def forward(self, x):
#         z = self.fc(x.view(x.shape[0], -1))
#         z = z.view(z.size(0), 30, 2)  
#         return z

In [11]:
# class ArgoNet(torch.nn.Module):
#     """
#     CNN
#     """
#     def __init__(self, device):
#         super(ArgoNet, self).__init__() 
        
#         self.device = device        
#         # Linear regression for 1 agent 
#         self.conv = nn.Sequential(
#             nn.Conv1d(19, 12, 1),
#             nn.SELU(),
#         )
#         self.fc = nn.Linear(12 * 4, 30 * 2)   
    
#     def forward(self, x):
#         z = self.conv(x)
#         z = self.fc(z.view(z.shape[0], -1))
#         z = z.view(z.size(0), 30, 2)  
#         return z

In [12]:
class ArgoNet(torch.nn.Module):
    """
    RNN
    """
    def __init__(self, device):
        super(ArgoNet, self).__init__() 
        
        self.device = device        
        self.hidden_size = 256
        self.num_layers = 1
        self.bidir = True
        self.num_dir = 2 if self.bidir else 1
        self.rnn = nn.LSTM(8, self.hidden_size, self.num_layers, batch_first=True, bidirectional=self.bidir)
        self.fc = nn.Linear(self.hidden_size * self.num_dir, 30 * 2)   
    
    def forward(self, x):
        z, (h, c) = self.rnn(x)
        out = self.fc(z[:, -1, :])
        out = out.view(out.size(0), 30, 2)  
        return out

In [13]:
# class ArgoNet(torch.nn.Module):
#     """
#     CNN + RNN
#     """
#     def __init__(self, device):
#         super(ArgoNet, self).__init__() 
        
#         self.device = device        
#         self.hidden_size = 120
#         self.num_layers = 1
#         self.bidir = True
#         self.num_dir = 2 if self.bidir else 1
#         self.conv = nn.Sequential(
#             nn.Conv1d(19, 19, 1),
#             nn.SELU(),
#         )
#         self.rnn = nn.LSTM(4, self.hidden_size, self.num_layers, batch_first=True, bidirectional=self.bidir)
#         self.fc = nn.Linear(self.hidden_size * self.num_dir, 30 * 2)   

    
#     def forward(self, x):
#         z = self.conv(x)
#         z, (h, c) = self.rnn(z)
#         out = self.fc(z[:, -1, :])
#         out = out.view(out.size(0), 30, 2)  
#         return out

In [14]:
# class ArgoNet(torch.nn.Module):
#     """
#     Encoder-Decoder
#     """
#     def __init__(self, device):
#         super(ArgoNet, self).__init__() 
        
#         self.device = device        
#         # Linear regression for 1 agent 
#         self.hidden_size = 120
#         self.num_layers = 1
#         self.bidir = True
#         self.num_dir = 2 if self.bidir else 1
#         self.enc = nn.LSTM(4, self.hidden_size, self.num_layers, batch_first=True, bidirectional=self.bidir)
#         self.dec = nn.LSTM(4, self.hidden_size, self.num_layers, batch_first=True, bidirectional=self.bidir)
#         self.fc = nn.Linear(self.num_dir * self.hidden_size, 1 * 4)   
    
#     def forward(self, x):
#         z, (h, c) = self.enc(x)
#         pred = torch.zeros((x.shape[0], 30, 2), device=self.device)
#         out = x[:, -1, :].unsqueeze(1)
#         for t in range(30):
#             out, (h, c) = self.dec(out, (h, c))
#             out = self.fc(out)
#             pred[:, t, :] = (out.squeeze(1)[:, :2])
#         return pred

In [15]:
def save_model(path, model, optimizer, scheduler, epoch, loss, batch):
    to_save = {
        MODEL_STATE: model.state_dict(),
        OPTIMIZER_STATE: optimizer.state_dict(),
        SCHEDULER_STATE: scheduler.state_dict(),
        EPOCH_STATE: epoch,
        LOSS_STATE: loss,
        BATCH_STATE: batch,        
    }
    torch.save(to_save, path)
    
def load_model(path, model, optimizer, scheduler):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint[MODEL_STATE])
    optimizer.load_state_dict(checkpoint[OPTIMIZER_STATE])
    scheduler.load_state_dict(checkpoint[SCHEDULER_STATE])
    return checkpoint[EPOCH_STATE], checkpoint[LOSS_STATE], checkpoint[BATCH_STATE]

# Evaluation

In [16]:
# Initialize datasets and loaders
# train_data, test_data, val_data = get_data('single', 'not_extended')
train_data, test_data, val_data = get_data('tracked', 'extended')

In [17]:
train_dataset = ArgoverseDataset(train_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, 
                                           shuffle=True, collate_fn=collate_train_test, 
                                           num_workers=N_WORKERS, drop_last=True)
test_dataset = ArgoverseDataset(test_data)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE_TEST, 
                                          shuffle=False, collate_fn=collate_train_test, 
                                          num_workers=N_WORKERS)
val_dataset = ArgoverseDataset(val_data)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, 
                                         shuffle=False, collate_fn=collate_val,
                                         num_workers=N_WORKERS)

In [18]:
train_losses = []
test_losses = []

model = ArgoNet(DEVICE).to(DEVICE)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
# optimizer = torch.optim.Adagrad(model.parameters())
# optimizer = torch.optim.Adadelta(model.parameters())
# optimizer = torch.optim.RMSprop(model.parameters())
# optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.AdamW(model.parameters())

optimizer = torch.optim.Adam(model.parameters())

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, threshold=0.0001)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=600, step_size_down=600)

print(f"Number of Argo model parameters is {sum(p.numel() for p in model.parameters())}")

Number of Argo model parameters is 575548


In [28]:
# Reload a model
# RELOAD_PATH = ""
# model = ArgoNet(DEVICE).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)
# _, (train_losses, test_losses), _ = load_model('current_model_storage/.tar', model, optimizer, scheduler)

# num_params = sum(p.numel() for p in model.parameters())   
# print(f"Number of model parameters is {num_params}")

In [19]:
NUM_EPOCH = 5

In [21]:
for t in range(1, NUM_EPOCH + 1):
    train_loss = train(model, DEVICE, train_loader, optimizer, 1, train_data)
    test_loss = test(model, DEVICE, test_loader, test_data)      
#     scheduler.step(test_loss)  # Uncomment this if using ReduceLROnPLateau
    train_losses.append(train_loss)
    test_losses.append(test_loss)        
    epoch = len(train_losses)
    print(f'Epoch {len(train_losses)}: train_loss = {train_loss}, test_loss = {test_loss}')

    save_model(f'report_opt/lstm_adam_lane_epoch{epoch}.tar', model, optimizer, scheduler, 
               len(train_losses), (train_losses, test_losses), 256)
    validate(model, DEVICE, 
             val_loader, f'report_opt/lstm_adam_lane_epoch{epoch}.csv')


Epoch 6: train_loss = 1.6984282414918517, test_loss = 1.7233123880337307



Epoch 7: train_loss = 1.690778491390391, test_loss = 1.705115380290854



Epoch 8: train_loss = 1.6826818389245075, test_loss = 1.7020819781566217



Epoch 9: train_loss = 1.6775050369353284, test_loss = 1.6920467621311028



Epoch 10: train_loss = 1.673590361149872, test_loss = 1.6922676786130764


# Loss Visualization

In [ ]:
def visualize_loss(losses):
    """
    Plots the losses over each training iteration. 
    Assumes that each element of the 'losses' list corresponds to the loss after each batch of train()
    """
    t_iter = np.arange(1, len(losses) + 1, 1, dtype=int)
    ax = sns.scatterplot(x=t_iter, y=losses, alpha=0.5)    
    ax.set_xlabel('Batch iteration number')
    ax.set_ylabel('Root-mean-square loss')
    ax.set_title('Batch Iteration vs. Root-Mean-Square Loss')
    plt.savefig('lossViter')

# Ground Truth Comparison

In [ ]:
def visualize_predictions(model, device, loader):
    """
    Compares some randomly selected data samples to the model's predictions
    """
    model.eval()
    
    # Get a batch of data
    _, (inp, out, scene_idxs, agent_idxs, masks) = next(enumerate(loader))
    
    # Move tensors to chosen device
    inp, out = inp.to(device), out.to(device)
    
    # Sample number
    i = 0
    
    # Scene idx
    scene_idx = scene_idxs[i]
        
    # Get contiguous arrays of the ground truth output positions
    truth = target[i].cpu().detach().numpy()
    x = truth[:, 0]
    y = truth[:, 1] 
        
    # Get contiguous arrays of the prediction output positions
    output = model(inp)    
    pred = output[i].cpu().detach().numpy()
    xh = pred[:, 0]
    yh = pred[:, 0]    
    
    # Plot the ground truth and prediction positions
    fig, (ax) = plt.subplots(nrows=1, ncols=1, figsize=(3, 3))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('Scene ' + str(scene_idx))
    ax.scatter(x, y, label='Ground Truth')
    ax.scatter(xh, yh, label='Prediction')
    ax.legend()